In [ ]:
import tensorflow as tf
from keras.applications import VGG16, ResNet50
from keras.models import Model
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
data_dir='./data'
batch_size = 32
image_size = (224, 224)
validation_split = 0.25
inputs = tf.keras.Input(shape=(224, 224, 3))
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# Load the ResNet50 model with ImageNet weights
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Create the feature extractor
feature_extractor = Model(inputs=model.input, outputs=model.layers[-1].output)


# Load your images
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
data_dir,
validation_split=validation_split,
subset="training",
seed=42,
labels="inferred",
label_mode=None,
image_size=image_size,
batch_size=batch_size)

features_train = feature_extractor.predict(train_ds)

features_train_flattened = np.reshape(features_train, (features_train.shape[0], -1))


SSE = []

# Loop over different numbers of clusters
for cluster in range(1, 20):
    kmeans = KMeans(n_clusters=cluster, random_state=0).fit(features_train_flattened)
    SSE.append(kmeans.inertia_)

# kmeans = KMeans(n_clusters=10)
# kmeans.fit(features_train_flattened)

In [ ]:
# Determine the elbow in the plot

# Plot the sum of squared distances
plt.style.use("ggplot")
plt.figure(figsize=(8, 6))
plt.plot(range(1, 20), SSE)
plt.xlabel('Number of clusters')
plt.ylabel('Sum of squared distances')
plt.show()
print(SSE)


x = range(1, len(SSE)+1)

from kneed import KneeLocator
kn = KneeLocator(x, SSE, S=1.0, curve='convex', direction='decreasing')
print(kn.elbow)
kn.plot_knee()
kn.plot_knee_normalized()


In [ ]:
kmeans.fit(features_train_flattened)

In [ ]:
# Get the cluster assignments for each data point
cluster_assignments = kmeans.labels_

# Count the number of data points in each class
class_counts = np.zeros(kmeans.n_clusters)
for i in range(kmeans.n_clusters):
  class_counts[i] = np.sum(cluster_assignments == i)

# Plot the bar plot
plt.bar(range(kmeans.n_clusters), class_counts)
plt.title('Number of Data Points in Each Class')
plt.xlabel('Class Label')
plt.ylabel('Number of Data Points')
plt.show()